In [1]:
from commonfunctions import *

### Parameters

Annotation => filename ; y ; x ; x_min ; y_min ; x_max ; y_max ; category

In [2]:
IMAGE_SIZE = (256, 256)  # To resize all images to 256x256
DATASET_PATH = "TSRD_Train"  # Path to your dataset folder
ANNOTATION_FILE = "TSRD_Train_Annotation/TsignRecgTrain4170Annotation.txt"  # Path to the annotation file
TEST_DATASET_PATH = "TSRD_Test"  # Path to your test dataset folder
FAILED_TEST_IMAGES_PATH = "failed_test_images"  # Path to save failed test images
MODEL_PATH = "trained_model.pkl"  # File to save the modedl

### Helper functions

In [3]:
# Function to parse annotation file ========================================================
def parse_annotations(annotation_file):
    annotations = []
    with open(annotation_file, "r") as file:
        for line in file:
            parts = line.strip().split(";")
            filename = parts[0]
            y_axis = int(parts[1])
            x_axis = int(parts[2])
            bound_x_min = int(parts[3])
            bound_y_min = int(parts[4])
            bound_x_max = int(parts[5])
            bound_y_max = int(parts[6])
            label = int(parts[7])
            annotations.append({
                "filename": filename,
                "y_axis": y_axis,
                "x_axis": x_axis,
                "bound_x_min": bound_x_min,
                "bound_y_min": bound_y_min,
                "bound_x_max": bound_x_max,
                "bound_y_max": bound_y_max,
                "label": label
            })
    return annotations


# Function to load images =================================================================
def load_images(image_folder, annotations):
    images = []
    labels = []
    for ann in annotations:
        # Get the image path
        img_path = os.path.join(image_folder, ann["filename"])
        # Read the image
        img = io.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        images.append(img)
        labels.append(ann["label"])
    return images, labels

# get the acutal labels of the test dataset
def get_actual_labels(TEST_DATASET_PATH):
    actual_labels = []
    for filename in os.listdir(TEST_DATASET_PATH):
        label = int(filename.split("_")[0])
        actual_labels.append(label)
    return actual_labels

def load_test_images(image_folder):
    blind_test_images = []
    for filename in os.listdir(TEST_DATASET_PATH):
        img_path = os.path.join(TEST_DATASET_PATH, filename)
        img = io.imread(img_path)
        blind_test_images.append(img)
    return blind_test_images

# augmentation
def augment_image(image):
    augmented_images = []

    # Original image
    augmented_images.append(image)
    
    # Brightness
    bright = cv.convertScaleAbs(image, alpha=1.2, beta=30)  # Increase brightness
    augmented_images.append(bright)

    dark = cv.convertScaleAbs(image, alpha=0.8, beta=30)  # Decrease brightness
    augmented_images.append(dark)
    
    return augmented_images

### Image Preprocessing

In [4]:
from skimage.morphology import disk, opening
# Function to load and preprocess images =================================================
def preprocess_images(image_folder, annotations, image_size):

    # load images and labels
    print("Loading images...")
    images, labels = load_images(image_folder, annotations)
    print("Successfully loaded images")

    print("Preprocessing images...")
    processed_images = []
    new_labels = []

    for i in range(len(images)):
        ann = annotations[i]
        img = images[i]
        label = labels[i]

        # # Crop the bounding box
        cropped_img = img[ann["bound_y_min"] :ann["bound_y_max"] , ann["bound_x_min"] :ann["bound_x_max"]]

        # Convert to grayscale
        gray_image = cv.cvtColor(cropped_img, cv.COLOR_BGR2GRAY)

        # Apply Gaussian filter with sigma=3
        gaussian_blur = cv.GaussianBlur(gray_image, (5, 5), sigmaX=1, sigmaY=1)
        
        # Perform opening morphology using a disk-shaped structuring element
        structuring_element = disk(4)  # Disk of radius 4
        opened_image = opening(gaussian_blur, structuring_element)

        # Resize the image
        resized_img = cv.resize(opened_image, image_size, interpolation=cv.INTER_CUBIC)

        # crop to the region of interest
        if labels[i] == 22 or labels[i] == 24 or labels[i] == 25 or labels[i] == 26 or labels[i] == 27 or labels[i] == 55 or labels[i] == 56: 
            # crop the circluar region with center equals the center of the bounding box and radius equals the half of the bounding box width
            mask1 = np.zeros_like(resized_img)
            center = (int(resized_img.shape[1]/2), int(resized_img.shape[0]/2))
            radius = int(resized_img.shape[1]/2)
            mask1 = cv.circle(mask1, center, radius, (255, 255, 255), -1)
            masked_img = cv.bitwise_and(resized_img, mask1)
        else:
            # crop the triangular region with base equals the width of the bounding box and height equals the height of the bounding box and the base of the triangle is the button of the bounding box
            mask = np.zeros_like(resized_img)
            pts = np.array([[0, resized_img.shape[0]], [resized_img.shape[1], resized_img.shape[0]], [int(resized_img.shape[1]/2), 0]], np.int32)
            mask = cv.fillPoly(mask, [pts], (255, 255, 255))
            masked_img = cv.bitwise_and(resized_img, mask)
            
        # Augment images with low number of samples
        if label == 56:
            augment_images = augment_image(masked_img)
            for img in augment_images:
                processed_images.append(img)
                new_labels.append(label)
        else:
            processed_images.append(masked_img)
            new_labels.append(label)

    # # Convert to NumPy arrays
    processed_images = np.array(processed_images)
    new_labels = np.array(new_labels)

    return processed_images, new_labels


# Function to preprocess a single test image =============================================
def preprocess_test_image(img):

    # image resizing
    img = cv.resize(img, IMAGE_SIZE, interpolation=cv.INTER_CUBIC)

    # Convert to grayscale
    gray_image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Apply Gaussian filter with sigma=3
    gaussian_blur = cv.GaussianBlur(gray_image, (7, 7), sigmaX=1, sigmaY=1)
    
    # Perform opening morphology using a disk-shaped structuring element
    structuring_element = disk(4)  # Disk of radius 4
    opened_image = opening(gaussian_blur, structuring_element)
    
    return opened_image

### Feature Extraction

In [5]:
# Shape-Based Features (HOG) ===============================================================

# HOG
def extract_hog_features(image):
    hog_features = hog(image, 
                       orientations=9, 
                       pixels_per_cell=(32, 32), 
                       cells_per_block=(4, 4), 
                       block_norm='L2-Hys', 
                       visualize=False, 
                       feature_vector=True)
    return hog_features

### Train


In [6]:
def train ():
    
    # Parse annotations
    annotations = parse_annotations(ANNOTATION_FILE)

    # Load and preprocess images
    images, labels = preprocess_images(DATASET_PATH, annotations, IMAGE_SIZE)
    print("Succesfully preprocessed images")

    print("Extracting features...")
    # list of HOG features
    images_histograms = []

    for i in range(len(images)):
        img = images[i]
        # Apply hog features
        hog = extract_hog_features(img)
        images_histograms.append(hog)
    print("Successfully extracted features")

    # Train SVM classifier
    print("Training SVM classifier...")
    svm = SVC(C=0.001, kernel="linear", random_state=50)
    svm.fit(images_histograms, labels)
    print("Successfully trained the SVM classifier")

    return svm

### Predict


In [7]:
# function to predict the labels of the test dataset =================================================
def predict (svm):

    predicted_labels = []
    
    # Load test images
    blind_test_images = load_test_images(TEST_DATASET_PATH)
    # Actual labels
    actual_labels = get_actual_labels(TEST_DATASET_PATH)

    for i in range(len(blind_test_images)):
        img = blind_test_images[i]
        actual_label = actual_labels[i]

        # Preprocess the image
        preprocessed_test_image = preprocess_test_image(img)

        # Apply HOG features
        hog = extract_hog_features(preprocessed_test_image)

        # predict the label
        predicted_label = svm.predict(hog.reshape(1, -1))
        predicted_labels.append(predicted_label[0])

        # Save failed test images
        if predicted_label[0] != actual_label:
            cv.imwrite(f"{FAILED_TEST_IMAGES_PATH}/{actual_label}_{predicted_label[0]}_{i}.jpg", img)

    return predicted_labels


# function to predict the label of a single image amd return the label name ==========================
def predict_image(svm, img):

    # Preprocess the image
    preprocessed_test_image = preprocess_test_image(img)

    # Extract HOG features
    hog = extract_hog_features(preprocessed_test_image)

    # Predict the label
    predicted_label = svm.predict(hog.reshape(1, -1))

    # Map label to prediction
    label_map = {
        22: "Turn Left",
        24: "Turn Right",
        25: "Keep Left",
        26: "Keep Right",
        27: "Pedestrian Crossing",
        55: "No Entry",
        56: "Stop Sign"
    }

    return label_map.get(predicted_label[0], "Unknown")

### Run the Training phase and save the trained model 


In [8]:
import joblib

# Train the model
svm = train()

# Save the trained model
joblib.dump(svm, MODEL_PATH)
print("Model saved as "+ MODEL_PATH )

Loading images...
Successfully loaded images
Preprocessing images...
Succesfully preprocessed images
Extracting features...
Successfully extracted features
Training SVM classifier...
Successfully trained the SVM classifier
Model saved as trained_model.pkl


#### Main (predict test images)


In [9]:
from sklearn.metrics import classification_report, accuracy_score

# Main execution
if __name__ == "__main__":

    # Load the trained model
    try:
        svm = joblib.load(MODEL_PATH)
    except:
        print("Error: Model not found!")
        print("Please train the model first! from the previous cell")
        exit()

    # Predict the labels of the test dataset
    predicted_labels = predict(svm)

    # Get the actual labels
    actual_labels = get_actual_labels(TEST_DATASET_PATH)
    print ("actual_labels   ", actual_labels)
    print ("predicted_labels", predicted_labels)

    # Evaluate the classifier
    if len(actual_labels) == len(predicted_labels):
        accuracy = accuracy_score(actual_labels, predicted_labels)
        print(f"Test Accuracy: {accuracy * 100:.2f}%")
        print(classification_report(actual_labels, predicted_labels))
    else:
        print("Error: Length mismatch between filtered labels!")


actual_labels    [22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25

# GUI

In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2 as cv
import joblib


def load_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if not file_path:
        return

    try:
        img = cv.imread(file_path)
        if img is None:
            raise ValueError("Failed to load the image.")

        # Convert to RGB for displaying
        img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)

        # Update GUI with the image
        image_label.configure(image=img_tk)
        image_label.image = img_tk
        image_label.file_path = file_path

        file_name_label.config(text=f"Loaded: {file_path.split('/')[-1]}")

        predict_button.config(state=tk.NORMAL)
    except Exception as e:
        messagebox.showerror("Error", f"Could not load image: {e}")

def make_prediction():
    try:
        file_path = image_label.file_path
        img = cv.imread(file_path)

        # Predict the label
        label = predict_image(svm, img)

        # Display the result
        result_label.config(text=f"Prediction: {label}")
    except Exception as e:
        messagebox.showerror("Error", f"Prediction failed: {e}")

# Initialize GUI
root = tk.Tk()
root.title("Image Prediction")
root.geometry("500x700")

# Load SVM model
svm = joblib.load('trained_model.pkl')

# GUI components
frame = tk.Frame(root)
frame.pack(pady=10)

file_name_label = tk.Label(frame, text="No image loaded", font=("Arial", 14))
file_name_label.pack()

image_label = tk.Label(frame)
image_label.pack()

load_button = tk.Button(root, text="Load Image", command=load_image, font=("Arial", 12))
load_button.pack(pady=5)

predict_button = tk.Button(root, text="Predict", command=make_prediction, font=("Arial", 12), state=tk.DISABLED)
predict_button.pack(pady=5)

result_label = tk.Label(root, text="Prediction: None", font=("Arial", 14))
result_label.pack(pady=10)

# Run the GUI
root.mainloop()
